In [34]:
# ! pip install onnxruntime
# ! pip install llama-index[docs,openai] llama-index-vector-stores-chroma chromadb
# ! pip install llama-index llama-parse 


# Extract image from PDF - Without API KEY

There are many methods to extract image as following 

### 1. PyMuPDF

In [38]:
! pip install PyMuPDF


In [ ]:
import fitz  # PyMuPDF

# Open the PDF file
pdf_document = fitz.open("sample.pdf")

# Iterate through each page
for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    image_list = page.get_images(full=True)

    # Iterate through each image
    for img_index, img in enumerate(image_list):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Save the image
        with open(f"image_page{page_number+1}_{img_index+1}.png", "wb") as image_file:
            image_file.write(image_bytes)


#### 2. pdfimages(poppler)

#### 3. PyPDF2

In [40]:
! pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [ ]:
import PyPDF2

# Open the PDF file
with open("sample.pdf", "rb") as file:
    reader = PyPDF2.PdfFileReader(file)

    # Iterate through each page
    for page_number in range(reader.numPages):
        page = reader.getPage(page_number)
        if "/XObject" in page["/Resources"]:
            xObject = page["/Resources"]["/XObject"].getObject()
            for obj in xObject:
                if xObject[obj]["/Subtype"] == "/Image":
                    data = xObject[obj]._data

                    # Save the image
                    with open(f"image_page{page_number+1}_{obj[1:]}.jpg", "wb") as image_file:
                        image_file.write(data)


# Using APIs

#### Unstructured

#### Llama-Index

In [7]:
! pip install nest_asyncio


In [32]:
import os 
import nest_asyncio
nest_asyncio.apply()

In [33]:
import os

llama_api_key =  os.getenv("LAMMA_INDEX")

os.environ["LLAMA_CLOUD_API_KEY"] =llama_api_key

In [47]:
from llama_parse import LlamaParse
# Assuming 'aquery' is an asynchronous method provided by the library

# Initialize LlamaParse with 'image' result type
parser = LlamaParse(
    api_key=llama_api_key,  # Ensure your API key is set
    result_type="structured" , # 'structured' output includes detailed elements
    language='en',
    skip_diagonal_text=True,
    do_not_unroll_columns=True,
    disable_image_extraction=True
    # verbose=True
)

# Load and parse the PDF document
documents = parser.load_data("src/input/PDF/bain 1.pdf")




Started parsing the file under job_id 723a4549-fbb8-48ba-96f2-5d001acaeefa
Error while parsing the file 'src/input/PDF/bain 1.pdf': 'structured'


In [41]:
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key=llama_api_key,
    result_type="markdown"  # Try 'markdown' or 'json' as alternatives
  #  verbose=True
)

documents = parser.load_data("src/input/PDF/2023-Annual-Sustainability-Report-April-2024.pdf")


Started parsing the file under job_id 5a1f235f-f699-4f3a-9840-87d5156b673f


KeyboardInterrupt: 

.

In [36]:
documents[0]

IndexError: list index out of range

In [16]:
# Process extracted images
for doc in documents:
    if 'image' in doc.metadata:
        image_path = doc.metadata['image']
        print(f"Extracted image saved at: {image_path}")

In [28]:
from llama_parse import LlamaParse
import os
import nest_asyncio


parser = LlamaParse(
    api_key=llama_api_key,
    result_type="structured",  # Try 'markdown' or 'json' as alternatives
    verbose=True
)

documents = parser.load_data("src/input/PDF/bain 1-10.pdf")


Started parsing the file under job_id 52dd65f3-096d-49c9-ac8b-701a7a74ec2d
Error while parsing the file 'src/input/PDF/bain 1-10.pdf': 'structured'


In [25]:
documents

[Document(id_='a30d69d9-9c92-411b-95c6-1412039582a2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='   L\n2  |\n 21', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='302e33f4-d25c-4afc-87f0-b5918c9a995d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Arpan Sheth\n\nArpan Sheth in Mumbai and Washington, DC (arpan.sheth@bain.com)\n\n# -Pacific Technology, Vector, and Advanced Analytics\n\nSriwatsan Krishnan in Mumbai (sriwatsan.k@bain.com)\n\npractices, as well as the India Private Equity and Alternative I

## Note - Facing challenges to pull image via llama-index, Now we are moving to other library 

# Testing with text 

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


import onnxruntime 


In [ ]:

# Load documents
documents = SimpleDirectoryReader("src/input/PDF/").load_data()

# Node parser setup
text_splitter = SentenceSplitter(chunk_size=1024)

# Parse nodes (assumes documents contain text, tables, images separately)
nodes = []
for doc in documents:
    # Example to identify tables or images (assuming pre-tagged or file-types known)
    if doc.metadata.get('content_type') == 'table':
        node = TableNode(text=doc.text, metadata=doc.metadata)
    elif doc.metadata.get('content_type') == 'image':
        node = ImageNode(image_path=doc.metadata['image_path'], metadata=doc.metadata)
    else:  # Default to text
        chunks = text_splitter.get_nodes_from_documents([doc])
        nodes.extend(chunks)
        continue
    nodes.append(node) 

In [32]:
documents[2]

Document(id_='9d152e21-8b3c-451c-ab4e-fec0ab31f8d6', embedding=None, metadata={'page_label': '3', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src\\input\\PDF\\2023-Annual-Sustainability-Report-April-2024.pdf', 'file_type': 'application/pdf', 'file_size': 23435861, 'creation_date': '2025-03-17', 'last_modified_date': '2025-03-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Christoph, CEO\nExecutive Introduction\nBCG’s Annual Sustainability Report tells the story of our positive \nimpact on society. Through key metrics and our clients’ testimonia

In [28]:
type(nodes[5])

nodes[5]

TextNode(id_='5228716d-c4d5-420c-bd0b-cb5a93903ae4', embedding=None, metadata={'page_label': '6', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src\\input\\PDF\\2023-Annual-Sustainability-Report-April-2024.pdf', 'file_type': 'application/pdf', 'file_size': 23435861, 'creation_date': '2025-03-17', 'last_modified_date': '2025-03-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0789af39-fb3e-4e9c-9933-81be0d33fd89', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '6', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src

In [ ]:
for node in nodes:
    node
    print("-------")
    print(node.metadata)
    print("-------")
    print(node.excluded_embed_metadata_keys)
    print("------")
    print(node.text)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    

-------
{'page_label': '1', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src\\input\\PDF\\2023-Annual-Sustainability-Report-April-2024.pdf', 'file_type': 'application/pdf', 'file_size': 23435861, 'creation_date': '2025-03-17', 'last_modified_date': '2025-03-17'}
-------
['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date']
------
2023 ANNUAL SUSTAINABILITY REPORT
From Bold Ideas to 
Exponential Impact  
April 2024
+++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------
{'page_label': '2', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src\\input\\PDF\\2023-Annual-Sustainability-Report-April-2024.pdf', 'file_type': 'application/pdf', 'file_size': 23435861, 'creation_date': '2025-03-17', 'last_modified_date': '2025-03-17'}
-------
['file_name', 'file_type', '

In [5]:
for node in nodes:
    print(node)
    print("-----")

Node ID: 921dd53c-7f2e-4e96-bc02-3b4430e9ba6c
Text: 2023 ANNUAL SUSTAINABILITY REPORT From Bold Ideas to
Exponential Impact   April 2024
-----
Node ID: 7a842ec3-f34e-43e7-ac6e-7fb67b7273d6
Text: BOSTON CONSULTING GROUP PROMOTING RESPONSIBLE BUSINESS PRACTICES
|  2023 ANNUAL SUSTAINABILITY REPORT       2 Contents 03   Executive
Introduction 05   About This Report 15   Driving Social Impact 36
Protecting Our Planet 62   Empowering Our People 81   Promoting
Responsible   Business Practices 95   Appendix
-----
Node ID: 02e5a357-bf5e-4b7b-9c11-9244b182e9af
Text: Christoph, CEO Executive Introduction BCG’s Annual
Sustainability Report tells the story of our positive  impact on
society. Through key metrics and our clients’ testimonials,  this
report showcases the many ways in which we strive to live up to  our
purpose: unlocking the potential of those who advance the world. The
biggest themes for BCG and ...
-----
Node ID: 5de12d71-2388-492b-bae9-4aab8cd695d0
Text: BOSTON CONSULTING GROUP INT

In [9]:
nodes[1]

TextNode(id_='e4b08489-d397-4393-ad96-2ec89ecc3b68', embedding=None, metadata={'page_label': '2', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src\\input\\PDF\\2023-Annual-Sustainability-Report-April-2024.pdf', 'file_type': 'application/pdf', 'file_size': 23435861, 'creation_date': '2025-03-17', 'last_modified_date': '2025-03-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9943e4a7-1ed3-47df-ba50-5f226be7e83c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '2', 'file_name': '2023-Annual-Sustainability-Report-April-2024.pdf', 'file_path': 'c:\\Users\\Ai\\OneDrive\\Programm\\GitHub\\HF_Projects\\src

In [11]:
# Set up ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_storage")
chroma_collection = chroma_client.get_or_create_collection("document_chunks")

# Create vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Embed with OpenAI
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Create index
index = VectorStoreIndex(nodes, vector_store=vector_store, embed_model=embed_model)

# Optional: Persist index
index.storage_context.persist(persist_dir="./chroma_storage")


In [17]:
! pip install langchain_community

  Using cached langchain_community-0.3.19-py3-none-any.whl.metadata (2.4 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
Using cached langchain_community-0.3.19-py3-none-any.whl (2.5 MB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.8.1-py3-none-any.whl (30 kB)


In [ ]:
# Step 4: Implement RAG with similarity filtering
def retrieve_relevant_nodes(query, similarity_threshold=0.8):
    retriever = index.as_retriever(similarity_top_k=5)
    retrieved_nodes = retriever.retrieve(query)
    relevant_nodes = [node for node in retrieved_nodes if node.score and node.score >= similarity_threshold]
    return relevant_nodes

In [18]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI


In [ ]:
template = """
Based on the following information:
{context}
Answer the query: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "query"])


In [21]:
llm = OpenAI(model="gpt-3.5-turbo")
llm_chain = LLMChain(prompt=prompt, llm=llm)


C:\Users\Ai\AppData\Local\Temp\ipykernel_12448\1154614854.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model="gpt-3.5-turbo")
C:\Users\Ai\AppData\Local\Temp\ipykernel_12448\1154614854.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [22]:


# Step 5: Summarize and present the output
def generate_summary(query):
    relevant_nodes = retrieve_relevant_nodes(query)
    if not relevant_nodes:
        return "No relevant information found."
    context = " ".join([node.text for node in relevant_nodes])
    response = llm_chain.run({"context": context, "query": query})
    return response




In [23]:
# Example usage
query = "What is sustinable future ?"
summary = generate_summary(query)
print(summary)

C:\Users\Ai\AppData\Local\Temp\ipykernel_12448\1526715088.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"context": context, "query": query})


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [26]:
LLAMA_CLOUD_API_KEY= "llx-Us7CkW2kZn79na85NFRb779FBg1BHOHcgb8SEuhH0LnnijWG"

In [27]:
# set up parser
parser = LlamaParse( api_key= LLAMA_CLOUD_API_KEY,
    result_type="markdown"  # "markdown" and "text" are available
)


In [28]:
# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}



In [36]:
import nest_asyncio
nest_asyncio.apply()

# Now run your original code
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['src/input/annual-review-2024-en.pdf'],
    file_extractor=file_extractor
).load_data()


Started parsing the file under job_id cfb7d987-f85d-4aa8-a5c0-6e377aca3eff


In [37]:
print(documents)

[Document(id_='4aa30b92-5dbd-478e-a041-06a57b5ccedd', embedding=None, metadata={'file_path': 'src\\input\\annual-review-2024-en.pdf', 'file_name': 'annual-review-2024-en.pdf', 'file_type': 'application/pdf', 'file_size': 12302024, 'creation_date': '2025-03-16', 'last_modified_date': '2025-03-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Annual Review 2024\n\n# NEScAFE ESPRESSO CONCENTRATE\n\n# rich Arabica Blend\n\nKlack\n\nSwed Vanilka\n\nNestlé.\n\n10 SERVINGS\n\n500 ML\n\nWe unlock the power of food to enhance quality of life for everyone, today and for generations to come.', path=None, url=None, mimetype=None), image_resource

In [38]:
# one extra dep
from llama_index.core import VectorStoreIndex

# create an index from the parsed markdown
index = VectorStoreIndex.from_documents(documents)

# create a query engine for the index
query_engine = index.as_query_engine()

# query the engine
query = "List all financials performance"
response = query_engine.query(query)
print(response)

Group sales: CHF 91.4 billion
Organic growth: +2.2%
Real internal growth: +0.8%
Underlying trading operating profit: CHF 15.7 billion
Underlying trading operating profit margin: 17.2%
Trading operating profit: CHF 14.6 billion
Trading operating profit margin: 16.0%
Earnings per share: CHF 4.19
Underlying earnings per share: +2.5%
Operating cash flow: CHF 16.7 billion
Free cash flow: CHF 10.7 billion
Proposed dividend: CHF 3.05
Proposed dividend increase: +1.7%


In [40]:
# query the engine
query = "List all businesses in bullet "
response = query_engine.query(query)
print(response)

- Coffee
- Cocoa
- Malt
- Other beverage


In [35]:
# query the engine
query = "What is the coorporate registered Address?"
response = query_engine.query(query)
print(response)

The corporate registered address is Bombay House, 24, Homi Mody Street, Mumbai 400001, Maharashtra, India.


In [45]:
# query the engine
query = "Key Non financial performance in bullet points only the headers and respective values only"
response = query_engine.query(query)
print(response)

- Climate: 20.38% net reduction of GHG emissions versus 2018 baseline
- Nature: 93.5% assessed deforestation-free for stated primary supply chains
- Packaging: 21.3% virgin plastic reduction versus 2018 baseline
- Diversity, equity and inclusion: 47.4% management positions held by women
- Youth: 10.2 million young people around the world with access to economic opportunities since 2017
- Health and Nutrition: 132.0 billion servings of affordable nutrition with micronutrient fortification provided in 2024


In [46]:
# query the engine
query = "can you identify the images in this document?"
response = query_engine.query(query)
print(response)

I cannot identify the images in the document based on the provided context information.
